## Review dashboard

https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48

## Get desired data endpoint from API

https://www.eia.gov/opendata/browser/electricity/rto/fuel-type-data

## Initialize route in Python

In [1]:
from eia import EIAClient

client = EIAClient() 
route = client.get_data_endpoint('electricity/rto/fuel-type-data')

2025-10-10 17:13:43,391 - INFO - EIAClient initialized.
2025-10-10 17:13:43,391 - INFO - Directly accessing data endpoint metadata for: electricity/rto/fuel-type-data
2025-10-10 17:13:43,391 - INFO - Fetching metadata for route: electricity/rto/fuel-type-data


https://www.eia.gov/opendata/browser/electricity/rto/region-sub-ba-data?frequency=hourly&data=value;&sortColumn=period;&sortDirection=desc;

## Filtering options

In [30]:
route.get?

Signature:
route.get(
    data_columns: Optional[List[str]] = None,
    facets: Optional[Dict[str, Union[str, List[str]]]] = None,
    frequency: Optional[str] = None,
    start: Optional[str] = None,
    end: Optional[str] = None,
    sort: Optional[List[Dict[str, str]]] = None,
    length: Optional[int] = None,
    offset: Optional[int] = None,
    output_format: Optional[Literal['json', 'xml']] = 'json',
    paginate: bool = True,
) -> pandas.core.frame.DataFrame
Docstring:
Retrieves data from this endpoint, stores it and metadata internally,
and returns the data as a pandas DataFrame. Handles pagination automatically by default.

Args:
    data_columns: List of data column IDs to retrieve. If None, all available columns are fetched.
    facets: Dictionary of facet filters, e.g., {'stateid': 'CA', 'sectorid': ['RES', 'COM']}
    frequency: Data frequency ID (e.g., 'daily', 'monthly')
    start: Start date/period
    end: End date/period
    sort: List of sort specifications
    leng

### Facets: column values

In [31]:
route.facets

FacetContainer(facets=[respondent, fueltype])

In [49]:
facet_options = {}

for name, facet in route.facets.items():
    values = facet.get_values()
    options = []
    for meta in values:
        options.append(meta.__dict__)
    
    facet_options[name] = options

2025-10-10 19:54:30,521 - INFO - Fetching facet values for facet 'respondent' in route: electricity/rto/fuel-type-data
2025-10-10 19:54:31,576 - INFO - Fetching facet values for facet 'fueltype' in route: electricity/rto/fuel-type-data


In [52]:
import yaml

with open("facet_options.yml", "w") as f:
    yaml.dump(facet_options, f, sort_keys=False)

### Frequencies: datetime intervals

In [53]:
pd.DataFrame(route.frequencies)

,id,description,query,format
0,hourly,One data point for each hour in UTC time.,H,"YYYY-MM-DD""T""HH24"
1,local-hourly,One data point for each hour in local time.,LH,"YYYY-MM-DD""T""HH24TZH"


## Query data

In [10]:
df = route.get(
    data_columns=['value'],
    facets={'respondent': 'PJM'},
    frequency='local-hourly',
    start='2024-01-01T00-05',
    end='2025-01-01T00-05',
    sort=[
        {'column': 'period','direction': 'desc'},
        {'column': 'fueltype','direction': 'asc'}
    ]
)

2025-10-10 17:17:45,897 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 0, Length: All (Pagination)
2025-10-10 17:17:45,898 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-10-10 17:17:48,446 - INFO - Received 5000 rows.
2025-10-10 17:17:48,447 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 5000, Length: All (Pagination)
2025-10-10 17:17:48,447 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-10-10 17:17:49,254 - INFO - Received 5000 rows.
2025-10-10 17:17:49,255 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 10000, Length: All (Pagination)
2025-10-10 17:17:49,255 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-10-10 17:17:52,589 - INFO - Received 5000 rows.
2025-10-10 17:17:52,590 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 15000, Length: All (Pagination)
2025-10-10 17:17:52,590 - INFO - Fetchin

## Preprocess data

In [17]:
import pandas as pd

In [24]:
x = pd.to_datetime(df.period, utc=True)
x

0       2025-01-01 05:00:00+00:00
1       2025-01-01 05:00:00+00:00
                   ...           
70268   2024-01-01 05:00:00+00:00
70269   2024-01-01 05:00:00+00:00
Name: period, Length: 70270, dtype: datetime64[ns, UTC]

In [25]:
x = x.dt.tz_convert('America/New_York')
x

0       2025-01-01 00:00:00-05:00
1       2025-01-01 00:00:00-05:00
                   ...           
70268   2024-01-01 00:00:00-05:00
70269   2024-01-01 00:00:00-05:00
Name: period, Length: 70270, dtype: datetime64[ns, America/New_York]

In [28]:
df.period = x
df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2025-01-01 00:00:00-05:00,PJM,"PJM Interconnection, LLC",COL,Coal,13878.0,megawatthours
1,2025-01-01 00:00:00-05:00,PJM,"PJM Interconnection, LLC",NG,Natural Gas,35650.0,megawatthours
...,...,...,...,...,...,...,...
70268,2024-01-01 00:00:00-05:00,PJM,"PJM Interconnection, LLC",WAT,Hydro,1354.0,megawatthours
70269,2024-01-01 00:00:00-05:00,PJM,"PJM Interconnection, LLC",WND,Wind,4020.0,megawatthours


## Export data

In [ ]:
df.to_parquet('../fuel_type_data_pjm.parquet')